In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import pandas as pd
import cv2
from skimage import io
from skimage import color
from skimage.feature import hog
from skimage.transform import resize
from skimage import io, transform
from multiprocessing import Pool
import pytesseract
from sklearn.model_selection import train_test_split

In [11]:
import pytesseract
import cv2
import re
from skimage import color

def detect_text(img_arr):
    for category in CATEGORIES:
        folder = os.path.join(Orignal_Folder, category)
        label = CATEGORIES.index(category)
        for img in os.listdir(folder):
            if not img.endswith('.jpg') and not img.endswith('.jpeg') and not img.endswith('.png'):
                # skip non-image files
                continue
            img_path = os.path.join(folder,img)
            try:
                img_arr = io.imread(img_path)
            except:
                # handle the case where the image cannot be read
                print(f"Error reading image file: {img_path}")
                continue

            # check if the image is grayscale or has an alpha channel
            if len(img_arr.shape) == 2 or img_arr.shape[2] == 4:
                # convert grayscale to RGB by replicating the grayscale channel three times
                img_arr = np.stack((img_arr,) * 3, axis=-1)

            # convert the image to uint8 data type
            img_arr = img_arr.astype(np.uint8)

            # resize the image to a consistent shape
            new_shape = (342,245, img_arr.shape[2])
            img_arr = transform.resize(img_arr, new_shape, anti_aliasing=True)

            gray = color.rgb2gray(img_arr)

    # Use Tesseract to extract text from the image
    text = pytesseract.image_to_string(blur, config='--psm 11')

    # Check the HP and attack values
    hp_match = re.search(r'^(\d+)\s+HP', text)
    attack_match = re.search(r'^(.+)\s(\d+)', text)
    if hp_match and attack_match:
        hp = int(hp_match.group(1))
        attack_no = int(attack_match.group(2))
        if hp > 300 or attack_no == 0 or attack_match.group(1).startswith(str(attack_no)):
            return False

    # Check the font size and style
    font_size = re.search(r'\d+px', text)
    font_style = re.search(r'font-family:\s*(.*?);', text, flags=re.IGNORECASE)
    if font_size and font_style:
        if int(font_size.group(0).strip('px')) < 30 or 'bold' not in font_style.group(1).lower():
            return False

    # Check the spelling of Pokemon names and attack names
    pokemon_name = re.search(r'^(.*)\n\n\s+(basic|stage \d)', text)
    if pokemon_name:
        pokemon_name = pokemon_name.group(1).strip()
        if any(char.isdigit() for char in pokemon_name):
            return False
        if 'é' not in pokemon_name:
            return False
    else:
        return False

    attack_names = re.findall(r'\n\n(.+)\n', text)
    if attack_names:
        for name in attack_names:
            if any(char.isdigit() for char in name):
                return False
            if 'é' not in name:
                return False
    else:
        return False

    return True

In [12]:
import cv2
import numpy as np

def detect_color(img_arr):
    # Load the image and preprocess it
    for category in CATEGORIES:
        folder = os.path.join(Orignal_Folder, category)
        label = CATEGORIES.index(category)
        for img in os.listdir(folder):
            if not img.endswith('.jpg') and not img.endswith('.jpeg') and not img.endswith('.png'):
                # skip non-image files
                continue
            img_path = os.path.join(folder,img)
            try:
                img_arr = io.imread(img_path)
            except:
                # handle the case where the image cannot be read
                print(f"Error reading image file: {img_path}")
                continue

            # check if the image is grayscale or has an alpha channel
            if len(img_arr.shape) == 2 or img_arr.shape[2] == 4:
                # convert grayscale to RGB by replicating the grayscale channel three times
                img_arr = np.stack((img_arr,) * 3, axis=-1)

            # convert the image to uint8 data type
            img_arr = img_arr.astype(np.uint8)

            # resize the image to a consistent shape
            new_shape = (342,245, img_arr.shape[2])
            img_arr = transform.resize(img_arr, new_shape, anti_aliasing=True)

            gray = color.rgb2gray(img_arr)
            blur = cv2.GaussianBlur(gray, (3,3), 0)

    # Check the color accuracy
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_color = np.array([0, 60, 50])
    upper_color = np.array([35, 255, 255])
    mask = cv2.inRange(hsv, lower_color, upper_color)
    color_accuracy = cv2.countNonZero(mask) / (image.shape[0] * image.shape[1])
    if color_accuracy < 0.5:
        return "Fake"

    # Check if the colors are too dark or faded
    gray_pixels = np.count_nonzero(blur < 200)
    total_pixels = image.shape[0] * image.shape[1]
    if gray_pixels / total_pixels > 0.2:
        return "Fake"

    # Check if the colors are smudged or inaccurate
    edged = cv2.Canny(blur, 30, 150)
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 200:
            rect = cv2.boundingRect(contour)
            x, y, w, h = rect
            if w > h:
                if w / h > 1.5:
                    return "Fake"
            else:
                if h / w > 1.5:
                    return "Fake"

    # Check if the colors are too bright or oversaturated
    brightness = np.sum(hsv[:, :, 2]) / (image.shape[0] * image.shape[1])
    saturation = np.sum(hsv[:, :, 1]) / (image.shape[0] * image.shape[1])
    if brightness > 150 or saturation > 150:
        return "Fake"

    return "Real"


In [13]:
def detect_shape_size(img_arr):
    # Get the dimensions of the image
    height, width, _ = image.shape

    # Check if the image is at least 2.5 x 3.5 inches
    if width < 2.5 * 300 or height < 3.5 * 300:
        return False

    # Check if the image is within the standard size of a Pokemon card
    standard_width = 6.3 * 300
    standard_height = 8.8 * 300
    if width < standard_width * 0.9 or width > standard_width * 1.1:
        return False
    if height < standard_height * 0.9 or height > standard_height * 1.1:
        return False

    return True


In [16]:
DIRECTORY = 'F:\Pokemon Cards Data set'

CATEGORIES = ["real"]

In [17]:
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split

def svm_model(img_arr, label):
    features = []
    labels = []
    text = detect_text(img_arr)
    Color = detect_color(img_arr)
    shape_and_size = detect_shape_size(img_arr)
    features.append([text, Color, shape_and_size])
    labels.append(label)
         
    return features, labels

# Load your image and convert it to a numpy array
image = Image.open(r'F:/Pokemon Cards Data set/Abomasnow_19_dp2.jpg')
img_arr = np.array(img)

# Call the svm_model function to extract features and labels from the image
features, labels = svm_model(img_arr, label)

# Convert the lists to numpy arrays
X = np.array(features)
y = np.array(labels)

# Split the dataset into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("Training samples: ", len(X_train))
print("Validation samples: ", len(X_val))
print("Testing samples: ", len(X_test))


FileNotFoundError: [Errno 2] No such file or directory: 'F:/Pokemon Cards Data set/Abomasnow_19_dp2.jpg'

In [15]:

# Create the destination Train, Validation & Test folders
Orignal_Folder = 'F:\Pokemon Cards Data set'
train_dir = os.path.join(Orignal_Folder, "train")
validation_dir = os.path.join(Orignal_Folder, "validation")
test_dir = os.path.join(Orignal_Folder, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [ ]:
#import shutil
#Create the destination Train, Validation & Test folders

#original_folder = 'F:\Pokemon Cards Data set'
#train_dir = 'F:\Pokemon Cards Data set\train'
#validation_dir = 'F:\Pokemon Cards Data set\validation'
#test_dir = 'F:\Pokemon Cards Data set\test'


#Orignal_Folder = 'F:\Pokemon Cards Data set'
#train_dir = os.path.join(Orignal_Folder, "train")
#validation_dir = os.path.join(Orignal_Folder, "validation")
#test_dir = os.path.join(Orignal_Folder, "test")
